Pandas 从 0.13 版开始（2014 年 1 月）就引入了实验性工具，让用户可以直接运行 C 语言速度的操作，不需要十分费力地配置中间数组。它们就是 eval() 和 query() 函数，都依赖于 Numexpr 程序包

# 1-query() 与 eval() 的设计动机：复合代数式

In [1]:
# NumPy 与 Pandas 都支持快速的向量化运算
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(100000)
y = rng.rand(100000)
%timeit x + y

44.2 µs ± 532 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [2]:
# 比普通的 Python 循环或列表综合要快很多
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)),dtype=x.dtype, count=len(x))

19.8 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
# 在处理复合代数式（compound expression）问题时的效率比较低
mask = (x > 0.5) & (y < 0.5)

In [4]:
# 由于 NumPy 会计算每一个代数子式，因此这个计算过程等价于
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

每段中间过程都需要显式地分配内存。

如果 x 数组和 y 数 组非常大，这么运算就会占用大量的时间和内存消耗。

Numexpr 程序库可以让在不为中间过程分配全部内存的前提下，完成元素到元素的复合代数式运算。

In [5]:
import numexpr
mask_numexpr = numexpr.evaluate('(x>0.5) & (y<0.5)')
np.allclose(mask, mask_numexpr)

True

好处是，由于 Numexpr 在计算代数式时不需要为临时数组分配全部内存，因此计算比 NumPy 更高效，尤其适合处理大型数组。

# 2-用 Pandas.eval() 实现高性能运算
    
    Pandas 的 eval() 函数用字符串代数式实现了 DataFrame 的高性能运算

In [6]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

In [7]:
# 用普通的 Pandas 方法计算四个 DataFrame 的和
%timeit df1+df2+df3+df4

99.7 ms ± 347 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# 也可以通过 pd.eval 和字符串代数式计算并得出相同的结果
%timeit pd.eval('df1+df2+df3+df4')

47.1 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


eval() 版本的代数式比普通方法快一倍（而且内存消耗更少）

In [9]:
np.allclose(df1+df2+df3+df4, pd.eval('df1+df2+df3+df4'))

True

01--pd.eval()支持的运算

In [10]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3))) 
                           for i in range(5))

001-算术运算符
    
    pd.eval() 支持所有的算术运算符

In [11]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

002-比较运算符

    pd.eval() 支持所有的比较运算符，包括链式代数式

In [12]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4) 
result2 = pd.eval('df1 < df2 <= df3 != df4') 
np.allclose(result1, result2)

True

003-位运算符
    
    pd.eval() 支持 &（与）和|（或）等位运算符

In [13]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4) 
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)') 
np.allclose(result1, result2)

True

在布尔类型的代数式中使用 and 和 or 等字面值

In [14]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

004-对象属性与索引

    pd.eval() 可以通过 obj.attr 语法获取对象属性，通过 obj[index] 语法获取对象索引

In [15]:
result1 = df2.T[0] + df3.iloc[1] 
result2 = pd.eval('df2.T[0] + df3.iloc[1]') 
np.allclose(result1, result2)

True

目前 pd.eval() 还不支持函数调用、条件语句、循环以 及更复杂的运算

# 3-用 DataFrame.eval() 实现列间运算
    由于 pd.eval() 是 Pandas 的顶层函数，因此 DataFrame 有一个 eval() 方法可以做类似的运算.

In [16]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C']) 
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [17]:
# 用前面介绍的 pd.eval()，就可以通过下面的代数式计算这三列
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)") 
np.allclose(result1, result2)

True

In [18]:
# DataFrame.eval() 方法可以通过列名称实现简洁的代数式
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

注意:用列名称作为变量来计算代数式，结果同样是正确的。

01-用DataFrame.eval()新增列
    
    DataFrame.eval() 还可以创建新的列

In [19]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [20]:
# 用 df.eval() 创建一个新的列 'D'，然后赋给它其他列计算的值
df.eval('D = (A + B)/C',inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [21]:
# 修改已有的列
df.eval('D = (A - B) / C', inplace=True) 
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


02-DataFrame.eval() 使用局部变量
    
    DataFrame.eval() 方法还支持通过 @ 符号使用 Python 的局部变量。

In [22]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

@ 符号表示“这是一个变量名称而不是一个列名称”，可以灵活地用两个“命名空间”的资源计算代数式。

@ 符号只能在 DataFrame.eval() 方法中使用，而不能在 pandas.eval() 函数中使用，因为 pandas.eval() 函数只能获取一个（Python）命名空间的内容。

# 4-DataFrame.query() 方法
    DataFrame 基于字符串代数式的运算实现了另一个方法，被称为 query()

In [23]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)] 
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]') 
np.allclose(result1, result2)

True

In [24]:
# 对于这种过滤运算，你可以用 query() 方法
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [25]:
# 注意: query() 方法也支持用 @ 符号引用局部变量
Cmean = df['C'].mean() 
result1 = df[(df.A < Cmean) & (df.B < Cmean)] 
result2 = df.query('A < @Cmean and B < @Cmean') 
np.allclose(result1, result2)

True

# 5-性能决定使用时机
    在考虑要不要用这两个函数时，需要思考两个方面：计算时间和内存消耗，而内存消耗是更重要的影响因素。

In [26]:
# 每个涉及 NumPy 数组或 Pandas 的 DataFrame 的复合代数式都会产生临时数组
x = df[(df.A < 0.5) & (df.B < 0.5)]

In [27]:
# 基本等价于
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5 
tmp3 = tmp1 & tmp2
x = df[tmp3]

临时 DataFrame 的内存需求比你的系统内存还大（通常是几吉字节），那么最好还是使用 eval() 和 query() 代数式

In [28]:
df.values.nbytes

32000

在性能方面，即使你没有使用最大的系统内存，eval() 的计算速度也比普通方法快。